### 1. Importar librerías y cargar datos 

In [381]:
import numpy as np
import pandas as pd
import warnings
import pickle
import xgboost as xgb
from datetime import datetime
from sklearn.model_selection import train_test_split

warnings.filterwarnings('ignore')

# Cargar datos
data = pd.read_csv('../data/data.csv')


### 2: División en datos de entrenamiento y de inferencia 

In [382]:
# Mezclar los datos antes de dividir
data_sample = data.sample(frac=1, random_state=42).reset_index(drop=True)

# División en 80% train y 20% inferencia
train_size = int(0.8 * len(data_sample))
data_train = data_sample.iloc[:train_size].copy()  # 80% entrenamiento
data_inf = data_sample.iloc[train_size:].copy()  # 20% inferencia


### 3: Feature Engineering (Aplicado a Train e Inferencia)

In [383]:
# Función para obtener el periodo del día
def get_period_day(date):
    date_time = datetime.strptime(date, '%Y-%m-%d %H:%M:%S').time()
    
    if datetime.strptime("05:00", '%H:%M').time() <= date_time <= datetime.strptime("11:59", '%H:%M').time():
        return 'mañana'
    elif datetime.strptime("12:00", '%H:%M').time() <= date_time <= datetime.strptime("18:59", '%H:%M').time():
        return 'tarde'
    else:
        return 'noche'

# Función para determinar si es temporada alta
def is_high_season(fecha):
    fecha_año = int(fecha.split('-')[0])
    fecha = datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S')
    
    ranges = [
        (datetime.strptime('15-Dec', '%d-%b'), datetime.strptime('31-Dec', '%d-%b')),
        (datetime.strptime('1-Jan', '%d-%b'), datetime.strptime('3-Mar', '%d-%b')),
        (datetime.strptime('15-Jul', '%d-%b'), datetime.strptime('31-Jul', '%d-%b')),
        (datetime.strptime('11-Sep', '%d-%b'), datetime.strptime('30-Sep', '%d-%b'))
    ]
    
    for r_min, r_max in ranges:
        if r_min.replace(year=fecha_año) <= fecha <= r_max.replace(year=fecha_año):
            return 1
    return 0

# Función para calcular la diferencia en minutos
def get_min_diff(row):
    fecha_o = datetime.strptime(row['Fecha-O'], '%Y-%m-%d %H:%M:%S')
    fecha_i = datetime.strptime(row['Fecha-I'], '%Y-%m-%d %H:%M:%S')
    return (fecha_o - fecha_i).total_seconds() / 60

# Aplicar Feature Engineering a Train e Inferencia
for df in [data_train, data_inf]:
    df['period_day'] = df['Fecha-I'].apply(get_period_day)
    df['high_season'] = df['Fecha-I'].apply(is_high_season)
    df['min_diff'] = df.apply(get_min_diff, axis=1)
    df['delay'] = np.where(df['min_diff'] > 15, 1, 0)

In [388]:
data_inf

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,period_day,high_season,min_diff,delay,predicted_delay
54564,2017-01-11 18:00:00,215,SCEL,SCIE,LAN,2017-01-11 18:02:00,215,SCEL,SCIE,LXP,...,Miercoles,N,Grupo LATAM,Santiago,Concepcion,tarde,1,2.0,0,0
54565,2017-11-14 12:04:00,334,SCEL,SCFA,LAN,2017-11-14 12:19:00,334,SCEL,SCFA,LXP,...,Martes,N,Grupo LATAM,Santiago,Antofagasta,tarde,0,15.0,0,0
54566,2017-09-05 11:00:00,293,SCEL,SCCI,LAN,2017-09-05 11:15:00,293,SCEL,SCCI,LXP,...,Martes,N,Grupo LATAM,Santiago,Punta Arenas,mañana,0,15.0,0,0
54567,2017-12-18 20:30:00,352,SCEL,SCCF,LAN,2017-12-18 20:34:00,352.0,SCEL,SCCF,LXP,...,Lunes,N,Grupo LATAM,Santiago,Calama,noche,1,4.0,0,0
54568,2017-12-27 12:17:00,88,SCEL,SCAT,JAT,2017-12-27 12:17:00,88.0,SCEL,SCAT,JAT,...,Miercoles,N,JetSmart SPA,Santiago,Copiapo,tarde,1,0.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,2017-07-29 17:50:00,461,SCEL,SABE,LAN,2017-07-29 18:22:00,461,SCEL,SABE,LAN,...,Sabado,I,Grupo LATAM,Santiago,Buenos Aires,tarde,1,32.0,1,1
68202,2017-02-22 22:35:00,11,SCEL,MMMX,AMX,2017-02-22 22:42:00,11,SCEL,MMMX,AMX,...,Miercoles,I,Aeromexico,Santiago,Ciudad de Mexico,noche,1,7.0,0,0
68203,2017-10-26 12:50:00,752,SCEL,SBGR,LAN,2017-10-26 14:05:00,752,SCEL,SBGR,LAN,...,Jueves,I,Grupo LATAM,Santiago,Sao Paulo,tarde,0,75.0,1,1
68204,2017-01-22 13:35:00,28,SCEL,YSSY,QFU,2017-01-22 13:51:00,28,SCEL,YSSY,QFA,...,Domingo,I,Qantas Airways,Santiago,Sydney,tarde,1,16.0,1,1


### 4: Preparar los datos para XGBoost

In [384]:
# Crear variables dummy para entrenamiento
features_train = pd.concat([
    pd.get_dummies(data_train['OPERA'], prefix='OPERA'),
    pd.get_dummies(data_train['TIPOVUELO'], prefix='TIPOVUELO'), 
    pd.get_dummies(data_train['MES'], prefix='MES')
], axis=1)

target_train = data_train['delay']

# Dividir en train y test
x_train, x_test, y_train, y_test = train_test_split(features_train, target_train, test_size=0.33, random_state=42)

### 5: Entrenar XGBoost y guardar el modelo

In [385]:
# Calcular el balance de clases
n_y0 = len(y_train[y_train == 0])
n_y1 = len(y_train[y_train == 1])
scale = n_y0 / n_y1

# Entrenar el modelo con balanceo de clases
xgb_model = xgb.XGBClassifier(random_state=1, learning_rate=0.01, scale_pos_weight=scale)
xgb_model.fit(x_train, y_train)

# Guardar el modelo
with open("xgb_model.pkl", "wb") as file:
    pickle.dump(xgb_model, file)


### Paso 6: Feature Engineering para Inferencia

In [386]:
# Crear variables dummy para inferencia
features_inf = pd.concat([
    pd.get_dummies(data_inf['OPERA'], prefix='OPERA'),
    pd.get_dummies(data_inf['TIPOVUELO'], prefix='TIPOVUELO'), 
    pd.get_dummies(data_inf['MES'], prefix='MES')
], axis=1)

# Asegurar que las columnas en inferencia sean las mismas que en entrenamiento
features_inf = features_inf.reindex(columns=x_train.columns, fill_value=0)


### 7: Inferencia con el modelo guardado

In [387]:
# Cargar el modelo guardado
with open("xgb_model.pkl", "rb") as file:
    loaded_xgb_model = pickle.load(file)

# Hacer predicciones en inferencia
y_pred_inf = loaded_xgb_model.predict(features_inf)

# Agregar la predicción a los datos de inferencia
data_inf['predicted_delay'] = y_pred_inf

# Mostrar algunas predicciones
data_inf[['Fecha-I', 'OPERA', 'SIGLADES', 'TIPOVUELO', 'predicted_delay']].head()


,Fecha-I,OPERA,SIGLADES,TIPOVUELO,predicted_delay
54564,2017-01-11 18:00:00,Grupo LATAM,Concepcion,N,0
54565,2017-11-14 12:04:00,Grupo LATAM,Antofagasta,N,0
54566,2017-09-05 11:00:00,Grupo LATAM,Punta Arenas,N,0
54567,2017-12-18 20:30:00,Grupo LATAM,Calama,N,0
54568,2017-12-27 12:17:00,JetSmart SPA,Copiapo,N,1
